<div align="center">

# Assignment 6

---

**Author:** Matthew Walczyk  
**Date:** 9/30/2024  
**Modified By:** Matthew Walczyk  
**Description:** This program contains my week 6 exercises using pandas.

---
</div>

In [15]:
import pandas as pd
from scipy.stats import zscore

In [16]:
earthquakes = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/refs/heads/master/ch_04/exercises/earthquakes.csv')

filtered_quakes = earthquakes[(earthquakes['magType'] == 'mb') & (earthquakes['mag'] >= 4.9)]

print(filtered_quakes.head())

     mag magType           time                               place  tsunami  \
227 5.20      mb  1539389603790             15km WSW of Pisco, Peru        0   
229 4.90      mb  1539389546300        193km N of Qulansiyah, Yemen        0   
248 4.90      mb  1539382925190  151km S of Severo-Kuril'sk, Russia        0   
258 5.10      mb  1539380306940       236km NNW of Kuril'sk, Russia        0   
391 5.10      mb  1539337221080             Pacific-Antarctic Ridge        0   

                parsed_place  
227                     Peru  
229                    Yemen  
248                   Russia  
258                   Russia  
391  Pacific-Antarctic Ridge  


In [17]:
# I made a copy because I was getting a warning about setting a value on a copy of a slice from a DataFrame
ml_quakes = earthquakes[earthquakes['magType'] == 'ml'].copy()

bins = range(int(ml_quakes['mag'].min()), int(ml_quakes['mag'].max()) + 2)  # Think this needs to be +2 to include the max value

ml_quakes.loc[:, 'magnitude_bin'] = pd.cut(ml_quakes['mag'], bins=bins, right=False)

magnitude_counts = ml_quakes['magnitude_bin'].value_counts().sort_index()

print(magnitude_counts)

magnitude_bin
[-1, 0)     446
[0, 1)     2072
[1, 2)     3126
[2, 3)      985
[3, 4)      153
[4, 5)        6
[5, 6)        2
Name: count, dtype: int64


In [18]:
faang = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/refs/heads/master/ch_04/exercises/faang.csv')

aggregations = {
    'open': 'mean',
    'high': 'max',
    'low': 'min',
    'close': 'mean',
    'volume': 'sum'
}

# Doing this because I did not want to see scientific notation
pd.options.display.float_format = '{:.2f}'.format

print(faang.agg(aggregations))

open             659.20
high            2050.50
low               36.65
close            658.60
volume   45832459300.00
dtype: float64


In [19]:
print(pd.crosstab(
    earthquakes['tsunami'], 
    earthquakes['magType'], 
    values=earthquakes['mag'], 
    aggfunc='max'
))

magType   mb  mb_lg   md   mh   ml  ms_20   mw  mwb  mwr  mww
tsunami                                                      
0       5.60   3.50 4.11 1.10 4.20    NaN 3.83 5.80 4.80 6.00
1       6.10    NaN  NaN  NaN 5.10   5.70 4.41  NaN  NaN 7.50


In [20]:
# Sorting to make sure the rolling window works
faang['date'] = pd.to_datetime(faang['date'])
faang = faang.sort_values(['ticker', 'date'])

# Not sure if we were supposed to drop the NaNs or not, but I did because it looked better
print(faang.groupby('ticker').rolling(window=60, on='date').agg({
    'open': 'mean',
    'high': 'max',
    'low': 'min',
    'close': 'mean',
    'volume': 'sum'
}).reset_index(drop=True).dropna())

       open   high    low  close        volume
59    43.09  45.88  37.56  43.07 9051611600.00
60    43.08  45.88  37.56  43.05 9102982000.00
61    43.05  45.88  37.56  43.02 9135257600.00
62    43.03  45.88  37.56  43.01 9166631200.00
63    43.00  45.88  37.56  42.99 9210413200.00
...     ...    ...    ...    ...           ...
1250 306.02 386.80 233.68 303.24  811001500.00
1251 303.60 386.80 231.23 301.23  818289300.00
1252 301.50 386.80 231.23 299.13  822147900.00
1253 299.39 380.93 231.23 297.12  824502000.00
1254 297.42 380.00 231.23 295.29  832212300.00

[960 rows x 5 columns]


In [21]:
print(faang.pivot_table(
    index='ticker', 
    values=['open', 'high', 'low', 'close', 'volume'], 
    aggfunc='mean'
))

         close    high     low    open       volume
ticker                                             
AAPL     47.26   47.75   46.80   47.28 136080258.17
AMZN   1641.73 1662.84 1619.84 1644.07   5648994.42
FB      171.51  173.61  169.30  171.47  27658596.81
GOOG   1113.23 1125.78 1101.00 1113.55   1741965.34
NFLX    319.29  325.22  313.19  319.62  11469624.70


In [22]:
amzn_q4_2018 = faang[(faang['ticker'] == 'AMZN') & 
                     (faang['date'] >= '2018-10-01') & 
                     (faang['date'] <= '2018-12-31')]

amzn_q4_2018_numeric = amzn_q4_2018[['open', 'high', 'low', 'close', 'volume']]

# Using the zscore function from scipy.stats
print(amzn_q4_2018_numeric.apply(zscore))

     open  high   low  close  volume
690  2.36  2.39  2.52   2.41   -1.64
691  2.21  2.25  2.27   2.17   -0.87
692  2.09  2.08  2.16   2.04   -0.93
693  1.86  1.83  1.80   1.74   -0.13
694  1.66  1.64  1.57   1.60   -0.30
..    ...   ...   ...    ...     ...
748 -2.20 -2.18 -2.21  -2.24   -0.14
749 -2.04 -1.62 -1.83  -1.35    1.13
750 -1.47 -1.65 -1.64  -1.42    0.86
751 -1.34 -1.34 -1.24  -1.30    0.50
752 -1.09 -1.28 -0.98  -1.13   -0.25

[63 rows x 5 columns]


In [23]:
events_data = {
    'ticker': ['FB', 'FB', 'FB'],
    'date': ['2018-07-25', '2018-03-19', '2018-03-20'],
    'event': [
        'Disappointing user growth announced after close.',
        'Cambridge Analytica story',
        'FTC investigation'
    ]
}

events_df = pd.DataFrame(events_data)

# Convert 'date' column to datetime
events_df['date'] = pd.to_datetime(events_df['date'])
events_df.set_index(['date', 'ticker'], inplace=True)

# Merge it!
print(pd.merge(faang, events_df, how='outer', on=['date', 'ticker']))

     ticker       date    high     low    open   close       volume event
0      AAPL 2018-01-02   43.08   42.31   42.54   43.06 102223600.00   NaN
1      AMZN 2018-01-02 1190.00 1170.51 1172.00 1189.01   2694500.00   NaN
2        FB 2018-01-02  181.58  177.55  177.68  181.42  18151900.00   NaN
3      GOOG 2018-01-02 1066.94 1045.23 1048.34 1065.00   1237600.00   NaN
4      NFLX 2018-01-02  201.65  195.42  196.10  201.07  10966900.00   NaN
...     ...        ...     ...     ...     ...     ...          ...   ...
1250   AAPL 2018-12-31   39.84   39.12   39.63   39.44 140014000.00   NaN
1251   AMZN 2018-12-31 1520.76 1487.00 1510.80 1501.97   6954500.00   NaN
1252     FB 2018-12-31  134.64  129.95  134.45  131.09  24625300.00   NaN
1253   GOOG 2018-12-31 1052.70 1023.59 1050.96 1035.61   1493300.00   NaN
1254   NFLX 2018-12-31  270.10  260.00  260.16  267.66  13508900.00   NaN

[1255 rows x 8 columns]


In [24]:
# Just making sure the data is in the right order
faang = faang.sort_values(['ticker', 'date'])

print(faang.groupby('ticker')[['open', 'high', 'low', 'close', 'volume']].transform(lambda x: x / x.iloc[0]))

      open  high  low  close  volume
251   1.00  1.00 1.00   1.00    1.00
252   1.01  1.01 1.02   1.00    1.16
253   1.01  1.01 1.02   1.00    0.88
254   1.02  1.02 1.02   1.02    0.93
255   1.02  1.02 1.03   1.01    0.80
...    ...   ...  ...    ...     ...
999   1.23  1.24 1.20   1.16    0.87
1000  1.19  1.26 1.18   1.26    1.31
1001  1.28  1.27 1.23   1.27    1.12
1002  1.32  1.30 1.28   1.27    1.00
1003  1.33  1.34 1.33   1.33    1.23

[1255 rows x 5 columns]
